# TAR: Proyecto Final 

In [ ]:
%pip install gymnasium pandas numpy openpyxl

## Tratamiento de datos

In [6]:
import numpy as np
import pandas as pd
import os

In [7]:
ruta_datosMOP = "./Datos/MOP"
ruta_datosADME = "./Datos/ADME"
ruta_datos_procesados = "./Datos_procesados"

## Entorno personalizado para el problema hidro-térmico

In [ ]:
import gymnasium as gym
from gymnasium import spaces

# to-do: Rodrigo aconsejo usar actorCritic (con one-hot encoding para las variables discretas) para las acciones continuas, si no converge discretizar el volumen del turbinado (ejemplo 10 niveles) y usar metodos tabulares (QLearning)

class HydroThermalEnv(gym.Env):
    T_MAX = 104  # Número máximo de pasos (2 años, 52 semanas * 2)
    N_HIDRO = 5

    P_BON_MAX = 155
    P_BAY_MAX = 108
    P_PAL_MAX = 333
    P_SAL_MAX = 1890
    P_CLAIRE_MAX = P_BON_MAX + P_BAY_MAX + P_PAL_MAX + P_SAL_MAX

    P_SOLAR_MAX = 254
    P_EOLICO_MAX = 1584.7
    P_BIOMASA_MAX = 487.3

    # to-do: revisar si estos valores son correctos
    P_TERMICO_BAJO_MAX = 100
    P_TERMICO_ALTO_MAX = 300

    Q_BON_MAX = 680
    Q_BAY_MAX = 828
    Q_PAL_MAX = 1372
    Q_SAL_MAX = 4200
    Q_CLAIRE_MAX = Q_BON_MAX + Q_BAY_MAX + Q_PAL_MAX + Q_SAL_MAX

    V_BON_MAX = 8200
    V_BAY_MAX = 0
    V_PAL_MAX = 1300
    V_SAL_MAX = 1500
    V_CLAIRE_MAX = V_BON_MAX + V_BAY_MAX + V_PAL_MAX + V_SAL_MAX
    V_CLAIRE_MIN = 0

    K_CLAIRE = P_CLAIRE_MAX / Q_CLAIRE_MAX

    V0 = V_CLAIRE_MAX / 2

    # to-do: revisar si estos valores son correctos
    VALOR_EXPORTACION = 12.5  
    COSTO_TERMICO_BAJO = 100  
    COSTO_TERMICO_ALTO = 200  

    def _inicial_hidrologia(self): ...

    def _siguiente_hidrologia(self, tiempo, hidrologia_actual): ...
    
    def _aportes(self): return 0

    def _vertido(self, qt):
        return np.max(self.v - qt + self._aportes() - self.V_CLAIRE_MAX, 0)
    
    # to-do: re diseñar cuando estén los nuevos datos de demanda
    def _demanda(self):
        # Leer datos de demanda desde archivo
        energias_demandas = self.energias["Demanda"]
        if self.t < len(energias_demandas):
            return energias_demandas[self.t]
        else:
            raise ValueError("Tiempo fuera de rango para datos de demanda")
    
    # to-do: re diseñar cuando estén los nuevos datos de generación eolico
    def _gen_eolico(self):
        # Leer datos eólicos desde archivo
        energias_eolico = self.energias["Eolico"]
        if self.t < len(energias_eolico):
            return energias_eolico[self.t]
        else:
            raise ValueError("Tiempo fuera de rango para datos eólicos")

    # to-do: re diseñar cuando estén los nuevos datos de generación solar
    def _gen_solar(self):
        # Leer datos solares desde archivo
        energias_solar = self.energias["Solar"]
        if self.t < len(energias_solar):
            return energias_solar[self.t]
        else:
            raise ValueError("Tiempo fuera de rango para datos solares")

    # to-do: completar la función de generación de biomasa
    def _gen_bio(self): return 0

    def _gen_renovable(self):
        # Generación total de energías renovables no convencionales
        return self._gen_eolico() + self._gen_solar() + self._gen_bio()

    def _gen_termico_bajo(self, demanda_residual):
        if demanda_residual <= self.P_TERMICO_BAJO_MAX:
            return demanda_residual
        else:
            return self.P_TERMICO_BAJO_MAX

    def _gen_termico_alto(self, demanda_residual):
        if demanda_residual <= self.P_TERMICO_ALTO_MAX:
            return demanda_residual
        else:
            raise ValueError("Demanda residual excede la capacidad del térmico alto")

    def _despachar(self, qt):
        demanda_residual = self._demanda() - self._gen_renovable() - (self.K_CLAIRE * qt)
        energia_termico_bajo = 0
        energia_termico_alto = 0
        exportacion = 0

        if demanda_residual > 0:
            # Primero uso termico barato
            energia_termico_bajo = self._gen_termico_bajo(demanda_residual)
            demanda_residual -= energia_termico_bajo

            if demanda_residual > 0:
                energia_termico_alto = self._gen_termico_alto(demanda_residual)
                demanda_residual -= energia_termico_alto

        if demanda_residual < 0:
            exportacion = -demanda_residual

        # to-do: revisar si los valores de exportación y costos son correctos
        return exportacion * self.VALOR_EXPORTACION, energia_termico_bajo * self.COSTO_TERMICO_BAJO + energia_termico_alto * self.COSTO_TERMICO_ALTO

    def __init__(self):
        # Defino espacios de observación y acción
        self.observation_space = spaces.Dict({
            "volumen": spaces.Box(self.V_CLAIRE_MIN, self.V_CLAIRE_MAX, shape=()),
            "hidrologia": spaces.Discrete(self.N_HIDRO, start=0),
            "tiempo": spaces.Discrete(self.T_MAX, start=0)
        })
        # turbinar en [0, V_MAX] continuo
        self.action_space = spaces.Box(self.V_CLAIRE_MIN, self.V_CLAIRE_MAX, shape=())

        self.v = self.V0                    # Volumen inicial del embalse
        self.t = 0                          # Tiempo inicial
        self.h = self._inicial_hidrologia() # Estado hidrológico inicial

        # Cargar datos de energías
        self.data_energias = leer_archivo(f"Datos\\MOP\\ESemEoloSolDem.xlsx", header=1)
        self.energias = {}
        for col in self.data_energias.columns:
            self.energias[col] = self.data_energias[col].values

        # Cargar datos de matrices hidrológicas
        self.data_matrices_hidrologicas = leer_archivo(f"Datos\\Claire\\matrices_sem.csv", sep=",", header=0)
        self.data_matrices_hidrologicas = self.data_matrices_hidrologicas.iloc[:, 1:] # Quito la columna de semanas
        self.matrices_hidrologicas = {}
        for i in range(self.data_matrices_hidrologicas.shape[0]):
            array_1d = self.data_matrices_hidrologicas.iloc[i, :].values
            self.matrices_hidrologicas[i] = array_1d.reshape(5, 5) # type: ignore

    def reset(self, *, seed=None, options=None):
        self.v = self.V0
        self.t = 0
        self.h = self._inicial_hidrologia()
        info = {
            "volumen_inicial": self.v,
            "hidrologia_inicial": self.h,
            "tiempo_inicial": self.t
        }
        return self._get_obs(), info
    
    def step(self, action):
        # Validar que la acción esté en el espacio válido
        assert self.action_space.contains(action), f"Acción inválida: {action}. Debe estar en {self.action_space}"
        qt = float(action)
        
        # dinámica: v ← v − q − d + a
        self.v = self.v - qt - self._vertido(qt) + self._aportes()
        self.h = self._siguiente_hidrologia(self.t, self.h)
        self.t += 1

        # despacho: e_eolo + e_sol + e_bio + e_termico + e_hidro = dem + exp
        ingreso_exportacion, costo_termico = self._despachar(qt)

        # recompensa: −costo_termico + ingreso_exportacion
        reward = -costo_termico + ingreso_exportacion
        
        done = (self.t >= self.T_MAX)
        info = {
            "volumen": self.v,
            "tiempo": self.t,
            "hidrologia": self.h,
            "turbinado": qt,
            "vertido": self._vertido(qt),
            "ingreso_exportacion": ingreso_exportacion,
            "costo_termico": costo_termico,
        }
        return self._get_obs(), reward, done, False, info
    
    def render(self, mode='human'):
        if mode == 'human':
            print(f"Semana {self.t}/52:")
            print(f"  Volumen embalse: {self.v:.2f}/{self.V_CLAIRE_MAX}")
            print(f"  Estado hidrológico: {self.h}")
            print(f"  Porcentaje llenado: {(self.v/self.V_CLAIRE_MAX)*100:.1f}%")
            print("-" * 30)
        elif mode == 'rgb_array':
            # Retornar una imagen como array numpy para grabación
            pass
        elif mode == 'ansi':
            # Retornar string para mostrar en terminal
            return f"T:{self.t} V:{self.v:.1f} H:{self.h}"
        
    def _get_obs(self):
        # Mapeo de variables internas a observación del agente
        obs = {
            "volumen": self.v, 
            "hidrologia": self.h, 
            "tiempo": self.t
        }
        # Validar contra observation_space (opcional, útil para debug)
        assert self.observation_space.contains(obs), f"Observación inválida: {obs}. Debe estar en {self.observation_space}"
        return obs
    
class OneHotFlattenObs(gym.ObservationWrapper):
    def __init__(self, env):
        super().__init__(env)
        self.num_weeks = 52
        # 1 (volumen) + N_HIDRO + 52 (tiempo)
        dim = 1 + env.N_HIDRO + self.num_weeks # type: ignore
        self.observation_space = spaces.Box(0.0, 1.0, shape=(dim,), dtype=np.float32)

    def observation(self, obs):
        # normalizo volumen
        v_norm = obs["volumen"] / self.env.V_CLAIRE_MAX # type: ignore
        # one-hot hidrología
        h = obs["hidrologia"]
        hidro_oh = np.zeros(self.env.N_HIDRO, dtype=np.float32) # type: ignore
        hidro_oh[h] = 1.0
        # one-hot tiempo (semana del año)
        semana = obs["tiempo"] % self.num_weeks
        time_oh = np.zeros(self.num_weeks, dtype=np.float32)
        time_oh[semana] = 1.0

        return np.concatenate(([v_norm], hidro_oh, time_oh), axis=0)
    
## Auxiliares

# Leer archivo 
def leer_archivo(rutaArchivo, sep=None, header=0):
    if rutaArchivo.endswith('.xlsx') or rutaArchivo.endswith('.xls'):
        return pd.read_excel(rutaArchivo, header=header)
    else:
        return pd.read_csv(rutaArchivo, sep=sep, header=header, encoding='cp1252')

In [10]:
# Crear el entorno y envolverlo para usar observaciones one-hot
env = HydroThermalEnv()
env = OneHotFlattenObs(env)